<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
pip install --pre gql[all]

     |████████████████████████████████| 81kB 2.6MB/s 
     |████████████████████████████████| 194kB 13.9MB/s 
     |████████████████████████████████| 296kB 47.7MB/s 
     |████████████████████████████████| 1.3MB 46.5MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 143kB 41.8MB/s 
  Created wheel for gql: filename=gql-3.0.0a5-py2.py3-none-any.whl size=32709 sha256=abbe43c7a700227825381a2ef8933a5b7eda081b20327bdb752d2c57ec202a40
  Stored in directory: /root/.cache/pip/wheels/59/f4/5b/d4678b0d7c58b5b71cb44b849833ab4e6a0a9e4b6aad41b016
Successfully built gql


In [4]:
import requests
import json

In [64]:
api_key='MI6JGndpuW3Kyc9cIjj5of9AhZUKET4JlGdRsiBnGZjL2ofbChjG0ksWdjXrHvdRN3cHosvu6bkDcvdOUCc5A_m-IAciq5kpjSqAxt5euSYLzB0q0vvWqmuyrI86YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)

# print(json.loads(req.text))
dataset=json.loads(req.text)["reviews"]
x_test=[]
for i in range(len(dataset)):
  x_test.append(dataset[i]["text"])
x_test2=x_test.copy()
print(x_test)
print(x_test2)

["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']
["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']


In [6]:
df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
dat=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])

# print(data)

# Traitement

In [65]:
def processing(data):
  if (isinstance(data,str)):
    #MINUSCULES
    data=data.lower()
    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    data=tokenizer.tokenize(data)

    #STOPWORDS
    data=[w for w in data if not w in list(nltk.corpus.stopwords.words('english'))]
    #STEMMING
    stemmer=LancasterStemmer()
    data=[stemmer.stem(w) for w in data]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    data=[Word_Lemmatizer.lemmatize(w) for w in data]

  elif (isinstance(data,list)):
    for i in range(len(data)):
      data[i]=data[i].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      data[i]=tokenizer.tokenize(data[i])

    #STOPWORDS
    for i in range(len(data)):
      data[i]=[w for w in data[i] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      data[i]=[stemmer.stem(w) for w in data[i]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      data[i]=[Word_Lemmatizer.lemmatize(w) for w in data[i]]
    # print(data)

  else :
    #MINUSCULES
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=data[i][j].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=tokenizer.tokenize(data[i][j])

    #STOPWORDS
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[stemmer.stem(w) for w in data[i][j]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
    # print(data)

  return data

In [201]:
# dat=processing(dat)
# processing("This movie is a very bad one")

In [139]:
dat=transformWords(dat,defineVocab(dat))

In [48]:
from collections import defaultdict
def defineVocab(data):
  v=set()
  for i in range(len(data)):
    for j in range(len(data[i])):
      for h in range(len(data[i][j])):
        v.add(data[i][j][h])

  vocab = defaultdict(lambda: 0)
  vocab['<PAD>'] = 1

  j=2
  for i in v:
    vocab[i]=j
    j+=1
  return vocab
  # print(len(vocab))

In [114]:
voca=defineVocab(dat)
print(voca)

defaultdict(<function defineVocab.<locals>.<lambda> at 0x7f7187910440>, {'<PAD>': 1, 'understand': 2, 'hatch': 3, 'ein': 4, 'officialtil': 5, 'colorofviol': 6, 'mang': 7, 'max': 8, 'tayswift': 9, 'ord': 10, 'miss_tattoo': 11, 'visit': 12, 'damnit': 13, 'swe': 14, 'idiot': 15, 'tragedy': 16, 'vivekg86': 17, 'blood': 18, 'bear': 19, 'smil': 20, 'ted': 21, 'endors': 22, 'stripper': 23, 'tejano': 24, 'jel': 25, 'bech': 26, 'radford': 27, '00': 28, 'woot': 29, 'wudnt': 30, 'baawwngggg': 31, 'planet': 32, 'deez': 33, 'gold': 34, 'westney': 35, 'chriswolfe2007': 36, 'apothecaryjer': 37, 'ym': 38, 'kisluvk': 39, 'overitnow': 40, 'plagu': 41, 'danzero': 42, 'julieebaby': 43, 'heah': 44, 'lorigam': 45, 'mi': 46, 'workin': 47, 'dizzy': 48, 'memb': 49, 'died': 50, 'as': 51, 'sidekick': 52, 'tshirt': 53, 'cut': 54, 'mad': 55, 'nationwideclass': 56, 'nd': 57, 'camer': 58, 'ind': 59, 'slushy': 60, 'yoeyfreshy': 61, 'co': 62, 'frog': 63, 'wow': 64, 'bluff': 65, 'behav': 66, 'tmrw': 67, 'useless': 68, 

In [53]:
# df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
# df.to_csv("gdrive/MyDrive/ProjetNLP/vocab",index=False)

In [52]:
# pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Id").to_dict()

In [51]:
# vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab")
# print(vocab)

In [49]:
def transformWords(data,vocab):
  matrixData=[]
  maxLen=0
  if (isinstance(data,list)):
    if (len(data)>1):
      for i in range(len(data)):
        tmp=[]
        for j in data[i]:
          if (j not in vocab.keys()):
            tmp.append(0)
          else:
            tmp.append(vocab[j])
        matrixData.append(tmp)
    else:
      for i in data:
        if (i not in vocab.keys()):
          matrixData.append(0)
        else:
          matrixData.append(vocab[i])
  else:
    for i in range(len(data)):
      for j in range(len(data[i])):
        if (maxLen < len(data[i][j])):
          maxLen= len(data[i][j])

    for i in range(len(data)):
      for j in range(len(data[i])):
        tmp=[]
        for k in range(maxLen):
          if (k >= len(data[i][j])):
            tmp.append(vocab['<PAD>'])
          else:
            tmp.append(vocab[data[i][j][k]])
        matrixData.append(tmp)
  return matrixData

In [ ]:
data=transformWords(data)
datas=np.array([df.iloc[1000:2000][5],df.iloc[801000:802000][5]])
datas=processing(datas)
datas=transformWords(datas)

In [ ]:
data=np.array(data)
datas=np.array(datas)

In [50]:
def buildModelLSTM(vocab):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.LSTM(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [51]:
def buildModelSimpleRNN(vocab):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.SimpleRNN(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [ ]:
print(data)

[[3247 2503 1010 ...    1    1    1]
 [1111 2347  791 ...    1    1    1]
 [3224 4596  394 ...    1    1    1]
 ...
 [1920 1034 3096 ...    1    1    1]
 [2084 2253 2078 ...    1    1    1]
 [ 107  667 1282 ...    1    1    1]]


In [52]:
# model=buildModel()
# def fitModel(model,trainSize): #trainSize = taille des données prises en compte pour le fit (ex : si on crée le modèle en lui donnant 20 phrases, trainSize=20)
#Attention, ici on a configuré le y_train pour qu'il y ait une part égale de commentaires positifs/négatifs
def fitModel(data,model,negSize,posSize):
  y_train=np.concatenate([np.zeros((negSize,1)),np.ones((posSize,1))])
  model.compile(loss='binary_crossentropy',optimizer="adam")
  model.fit(data,y_train,epochs=50)

In [53]:
import os
def final_LSTM(sentence):
  threshold=0.2
  model_file="LSTM_Model"
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelLSTM(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
  # return model
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




In [54]:
import os
def final_SimpleRNN(sentence):
  threshold=0.4
  model_file="SimpleRNN_Model"
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelSimpleRNN(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
  # return model
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




In [66]:
# LSTM=final_LSTM("Very tidy and lovely AirBnb apartment equipped with everything you need. A good bed and nice bathroom. Paul is a great host and there when you need him, Very nice and wants to share all hé know about the area. We had a great stay!")
# print(LSTM)
final_LSTM(x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

In [67]:
# Simple_RNN=final_SimpleRNN("Very tidy and lovely AirBnb apartment equipped with everything you need. A good bed and nice bathroom. Paul is a great host and there when you need him, Very nice and wants to share all hé know about the area. We had a great stay!")
# print(Simple_RNN)
final_SimpleRNN(x_test2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

In [144]:
from sklearn.metrics import f1_score
def defineThreshold(datas,model):
  y_train=np.concatenate([np.zeros((1000,1)),np.ones((1000,1))])
  seuils=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
  F1_score=[]
  predict=model.predict(datas)
  for seuil in seuils:
    predictions=[]
    for i in range(len(datas)):
      if (predict[i]<seuil):
        predictions.append(0)
      else:
        predictions.append(1)
    F1_score.append(f1_score(y_train,predictions))
  print(F1_score)
  seuil=seuils[F1_score.index(max(F1_score))]
  print(seuil)
  return seuil

In [149]:
# defineThreshold(np.array(dat),Simple_RNN)

In [ ]:
# np.expand_dims(datas[i],axis=0)